In [11]:
library(cluster)
library(arules)

<div style="background-color: #FFFFE0; font-family: 'Times New Roman'; padding: 10px;">    

## Clasificar utilizando K-means
</div>

<div style="background-color: #FFFFE0; font-family: 'Times New Roman'; padding: 10px;">    
    <p>
        El código a continuación realiza las siguientes tareas:
    </p>
    <ul>
        <li>Carga el dataset limpio desde un archivo CSV llamado <code>datos_limpios.csv</code>.</li>
        <li>Define una función <code>normalizar</code> para normalizar una columna de datos dada.</li>
        <li>Define una función <code>obtener_rangos</code> que aplica clustering K-means a una columna especificada, normaliza la columna y asigna etiquetas ordenadas (Bajo, Medio, Alto) a los clusters basándose en los valores medios de los datos originales.</li>
        <li>Aplica la función <code>obtener_rangos</code> a una lista de columnas especificadas (<code>duracion_segundos</code>, <code>impresiones</code>, <code>vistas</code>, <code>suscriptores</code>) y muestra los rangos reales para cada categoría ordenada.</li>
    </ul>
</div>

In [12]:
# Cargar el dataset limpio desde un archivo CSV
df <- read.csv("../../datasets/datos_limpios.csv")

# Normalizar una columna
normalizar <- function(x) {
  return((x - min(x)) / (max(x) - min(x)))
}

obtener_rangos <- function(data, columna, k = 3) {
  set.seed(123) # Reproducibilidad

  # Normalizar la columna
  columna_norm <- normalizar(data[[columna]])

  # Aplicar K-means
  kmeans_result <- kmeans(columna_norm, centers = k)

  # Calcular el Silhouette Score
  sil_score <- silhouette(kmeans_result$cluster, dist(columna_norm))
  promedio_silhouette <- mean(sil_score[, 3])

  # Ordenar los clusters por la media de los valores originales
  orden_clusters <- order(tapply(data[[columna]], kmeans_result$cluster, mean))

  # Asignar etiquetas ordenadas: Bajo, Medio, Alto
  etiquetas <- c("Bajo", "Medio", "Alto")

  # Mostrar los rangos reales para cada categoría ordenada
  for (i in 1:k) {
    cluster_index <- orden_clusters[i]
    valores_cluster <- data[[columna]][kmeans_result$cluster == cluster_index]
    rango_min <- min(valores_cluster)
    rango_max <- max(valores_cluster)
    cat("Rango", etiquetas[i], ":", rango_min, "-", rango_max, "\n")
  }

  # Mostrar el Silhouette Score promedio
  cat("Silhouette Score Promedio:", promedio_silhouette, "\n")
}

# Aplicar y mostrar los rangos de cada columna seleccionada
columnas <- c("duracion_segundos", "impresiones", "vistas", "suscriptores")

for (columna in columnas) {
  cat("\nRangos para", columna, ":\n")
  obtener_rangos(df, columna)
}


Rangos para duracion_segundos :
Rango Bajo : 24 - 57 
Rango Medio : 195 - 323 
Rango Alto : 399 - 443 
Silhouette Score Promedio: 0.8006478 

Rangos para impresiones :
Rango Bajo : 235 - 2707 
Rango Medio : 3450 - 6190 
Rango Alto : 10523 - 11458 
Silhouette Score Promedio: 0.7605541 

Rangos para vistas :
Rango Bajo : 38 - 163 
Rango Medio : 212 - 278 
Rango Alto : 394 - 597 
Silhouette Score Promedio: 0.6596161 

Rangos para suscriptores :
Rango Bajo : 0 - 1 
Rango Medio : 2 - 3 
Rango Alto : 4 - 5 
Silhouette Score Promedio: 0.6776471 


<div style="background-color: #FFFFE0; font-family: 'Times New Roman'; padding: 10px;">    

## Clasificar utilizando la función discretize
</div>

<div style="background-color: #FFFFE0; font-family: 'Times New Roman'; padding: 10px;">    
    <p>
        El código a continuación tiene como propósito cargar un conjunto de datos desde un archivo CSV, agrupar sus valores continuos en categorías discretas. Esto se logra utilizando la función <code>discretize</code>, que permite dividir un rango continuo en intervalos predefinidos. A continuación, se detallan las operaciones y parámetros del código:
    </p>
    <ul>
        <li>
            <strong>Carga del conjunto de datos:</strong>  
            <p>El archivo <code>for_ranges.csv</code>
        </li>
        <li>
            <strong>Uso de la función <code>discretize</code>:</strong>  
            <p>La función <code>discretize</code> toma un vector de datos continuos y los transforma en categorías discretas, divididas en intervalos. Este proceso se conoce como discretización. En el presente caso se utiliza el método de discretización por intervalos (<code>method = "interval"</code>) en la mayoría de las columnas. Esto significa que los valores se agrupan en segmentos equidistantes según el rango total de los datos.</p>
        </li>
        <li>
            <strong>Columnas procesadas:</strong>  
            <ul>
                <li>
                    <code>duracion_promedio_vistas:</code>  
                    <p>Se discretiza utilizando el método de intervalos. La salida indica los rangos en que los valores de esta columna se agrupan.</p>
                </li>
                <li>
                    <code>vistas:</code>  
                    <p>Se discretiza sin especificar un método. En este caso, <code>discretize</code> usa un enfoque predeterminado para categorizar los valores.</p>
                </li>
                <li>
                    <code>tiempo_reproduccion_horas:</code>  
                    <p>Se discretiza mediante el método de intervalos, similar a <code>duracion_promedio_vistas</code>.</p>
                </li>
                <li>
                    <code>suscriptores:</code>  
                    <p>Los valores de esta columna también se agrupan en intervalos equidistantes.</p>
                </li>
                <li>
                    <code>impresiones:</code>  
                    <p>Se discretiza usando el método de intervalos, dividiendo el rango de valores en partes iguales.</p>
                </li>
                <li>
                    <code>ctr:</code>  
                    <p>La métrica de clics (CTR) se discretiza de forma similar a las otras columnas mencionadas.</p>
                </li>
            </ul>
        </li>
        <li>
            <strong>Parámetros clave de <code>discretize</code>:</strong>  
            <ul>
                <li><code>method</code>: Define el método de discretización. En este caso, se utiliza <code>"interval"</code>, que divide el rango de valores en intervalos de tamaño uniforme.</li>
                <li><code>data</code>: El vector de datos continuos que será discretizado. Aquí se procesan columnas específicas del conjunto de datos cargado.</li>
            </ul>
        </li>
        <li>
            <strong>Resultados:</strong>  
            <p>Para cada columna, el código imprime en la consola los intervalos o categorías resultantes, proporcionando información sobre cómo se agruparon los datos originales, en categorías discretas.</p>
        </li>
    </ul>
</div>

In [13]:
# create vector of values
my_values <- read.csv("../../datasets/for_ranges.csv")

# discretize values in vector and print attribute
cat("Rangos para duracion_promedio_vistas:\n")
discretize(my_values$duracion_promedio_vistas, method = "interval")

cat("Rangos para vistas:\n")
discretize(my_values$vistas)

cat("Rangos para tiempo_reproduccion_horas:\n")
discretize(my_values$tiempo_reproduccion_horas, method = "interval")

cat("Rangos para suscriptores:\n")
discretize(my_values$suscriptores, method = "interval")

cat("Rangos para impresiones:\n")
discretize(my_values$impresiones, method = "interval")

cat("Rangos para ctr:\n")
discretize(my_values$ctr, method = "interval")

Rangos para duracion_promedio_vistas:


[1] [27,66)   [105,144] [27,66)   [105,144] [105,144] [66,105)  [105,144]
 [8] [66,105)  [105,144] [66,105)  [105,144] [66,105)  [27,66)   [27,66)  
[15] [27,66)   [27,66)   [27,66)   [27,66)   [27,66)   [66,105)  [66,105) 
attr(,"discretized:breaks")
[1]  27  66 105 144
attr(,"discretized:method")
[1] interval
Levels: [27,66) [66,105) [105,144]

Rangos para vistas:


[1] [317,6.96e+03] [317,6.96e+03] [317,6.96e+03] [317,6.96e+03] [317,6.96e+03]
 [6] [317,6.96e+03] [317,6.96e+03] [150,317)      [150,317)      [150,317)     
[11] [150,317)      [150,317)      [150,317)      [150,317)      [72,150)      
[16] [72,150)       [72,150)       [72,150)       [72,150)       [72,150)      
[21] [72,150)      
attr(,"discretized:breaks")
[1]   72.0000  150.3333  316.6667 6961.0000
attr(,"discretized:method")
[1] frequency
Levels: [72,150) [150,317) [317,6.96e+03]

Rangos para tiempo_reproduccion_horas:


...

[1] [20,30] [10,20) [10,20) [10,20) [0,10)  [0,10)  [0,10)  [0,10)  [0,10) 
[10] [0,10)  [0,10)  [0,10)  [0,10)  [0,10)  [0,10)  [0,10)  [0,10)  [0,10) 
[19] [0,10)  [0,10)  [0,10) 
attr(,"discretized:breaks")
[1]  0 10 20 30
attr(,"discretized:method")
[1] interval
Levels: [0,10) [10,20) [20,30]

Rangos para impresiones:


[1] [1.79e+05,2.68e+05] [777,9e+04)         [777,9e+04)        
 [4] [777,9e+04)         [777,9e+04)         [777,9e+04)        
 [7] [777,9e+04)         [777,9e+04)         [777,9e+04)        
[10] [777,9e+04)         [777,9e+04)         [777,9e+04)        
[13] [777,9e+04)         [777,9e+04)         [777,9e+04)        
[16] [777,9e+04)         [777,9e+04)         [777,9e+04)        
[19] [777,9e+04)         [777,9e+04)         [777,9e+04)        
attr(,"discretized:breaks")
[1]    777.00  89989.67 179202.33 268415.00
attr(,"discretized:method")
[1] interval
Levels: [777,9e+04) [9e+04,1.79e+05) [1.79e+05,2.68e+05]

Rangos para ctr:


[1] [1.62,3.18) [3.18,4.73) [4.73,6.29] [3.18,4.73) [1.62,3.18) [3.18,4.73)
 [7] [1.62,3.18) [1.62,3.18) [1.62,3.18) [3.18,4.73) [3.18,4.73) [1.62,3.18)
[13] [1.62,3.18) [3.18,4.73) [4.73,6.29] [3.18,4.73) [4.73,6.29] [4.73,6.29]
[19] [1.62,3.18) [1.62,3.18) [3.18,4.73)
attr(,"discretized:breaks")
[1] 1.620000 3.176667 4.733333 6.290000
attr(,"discretized:method")
[1] interval
Levels: [1.62,3.18) [3.18,4.73) [4.73,6.29]